## "Are You Irish" model development notebook

TL;DR:
The model is a Complement Naive Bayes, and the grid search selection criteria was F2 because I wanted to err on the side of Recall and minimizing False Negatives, even at the cost of False Positives.

#### Import libraries

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import classification_report

import pickle

#### Build dataset from CSV files in Google Drive

In [2]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTqd_a5YgMTVUGfc6F9dx62fvq2ptPvyq6cGpfAtPMYWTDn1qiVg2_ma79xS2NyQ-CHkOXy_MzCd03I/pub?gid=712009217&single=true&output=csv')
# Note: The encoding is not removing spaces, apostrophes, or accents on characters
# I manually removed them for now. #TechDebt


# Import other countries
country_urls = ['https://docs.google.com/spreadsheets/d/e/2PACX-1vTqd_a5YgMTVUGfc6F9dx62fvq2ptPvyq6cGpfAtPMYWTDn1qiVg2_ma79xS2NyQ-CHkOXy_MzCd03I/pub?gid=1621398561&single=true&output=csv'
               ,'https://docs.google.com/spreadsheets/d/e/2PACX-1vTqd_a5YgMTVUGfc6F9dx62fvq2ptPvyq6cGpfAtPMYWTDn1qiVg2_ma79xS2NyQ-CHkOXy_MzCd03I/pub?gid=1688323099&single=true&output=csv'
               ,'https://docs.google.com/spreadsheets/d/e/2PACX-1vTqd_a5YgMTVUGfc6F9dx62fvq2ptPvyq6cGpfAtPMYWTDn1qiVg2_ma79xS2NyQ-CHkOXy_MzCd03I/pub?gid=311250649&single=true&output=csv' 
               ]

big_surname_list = []

# Multiply each name by the number of times it should be in the list (col index 5)
for url in country_urls:
    country_df = pd.read_csv(url, usecols=[1,5])
    country_surnames = country_df.values.tolist()

    for name in country_surnames:
        for qty in range(0,name[1]):
            big_surname_list.append(name[0])

# Create 0 flags for all new names
surname_flags = [0] * len(big_surname_list)

# Zip it all together and join into master dataframe
new_surnames = pd.DataFrame(list(zip(big_surname_list,surname_flags)), columns=['surname_plain','irish_flag'])
df = pd.concat([df, new_surnames])

#### Grid search based on F2 score

In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    df['surname_plain'], df['irish_flag'], test_size=0.15, random_state=123, stratify=df['irish_flag'])

pipe = Pipeline([('vec', CountVectorizer())
                ,('clf', ComplementNB())
                # ,('rf', RandomForestClassifier())
                # ,('knn', KNeighborsClassifier())
                ])
param_grid = [{'vec__ngram_range':[(3,3),(4,4),(5,5)]
              ,'vec__analyzer':['char','char_wb']
              ,'clf__alpha':[0.01, 0.1, 0.5, 1, 10]
              # ,'knn__n_neighbors':[1,2,3,4,5]
              # ,'knn__weights':['uniform','distance']
              }]
f2_scorer = make_scorer(fbeta_score, beta=2)


gs = GridSearchCV(pipe
                  ,param_grid=param_grid
                  ,cv=3
                  ,verbose=1
                  ,scoring=f2_scorer)
gs.fit(X_train,y_train)

print(f"Best model parameters: {gs.best_params_}")
final_model_params = gs.best_params_
print()
print(f"Best training F2 score: {gs.best_score_}")
print()
# print(gs.cv_results_['mean_test_score'],'\n',gs.cv_results_['std_test_score'])

df_pred = pd.DataFrame(zip(X_test,y_test,gs.predict_proba(X_test),gs.predict(X_test), gs.predict_log_proba(X_test)))

print(f"F2 score on test data: {gs.score(X_test, y_test)}")
print()
print("Classification Report")
print(classification_report(y_test, gs.predict(X_test)))



Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best model parameters: {'clf__alpha': 0.1, 'vec__analyzer': 'char_wb', 'vec__ngram_range': (5, 5)}

Best training F2 score: 0.8693676972849392

F2 score on test data: 0.8968347010550997

Classification Report
              precision    recall  f1-score   support

           0       0.97      0.93      0.95      1536
           1       0.80      0.92      0.86       497

    accuracy                           0.93      2033
   macro avg       0.89      0.93      0.90      2033
weighted avg       0.93      0.93      0.93      2033



#### Create final pipeline with tuned hyperparameters, train final model

In [4]:
final_pipe = Pipeline([('vec', CountVectorizer(analyzer='char_wb', ngram_range=(4,4)))
                      ,('clf', ComplementNB(alpha=0.1))
                      ])
                      
final_model = final_pipe.fit(df['surname_plain'], df['irish_flag'])

#### Get class `irish_flag=1` feature log probabilities, save for use in app

In [7]:
def get_salient_words(nb_clf, vect, class_ind):
    """Return salient words for given class
    Parameters
    ----------
    nb_clf : a Naive Bayes classifier (e.g. MultinomialNB, BernoulliNB)
    vect : CountVectorizer
    class_ind : int
    Returns
    -------
    list
        a sorted list of (word, log prob) sorted by log probability in descending order.

    Note: Feature log probabilities for Class 1 are obtained by calling all the feature log probabilities for Class 0 and  multiplying them by -1
    """

    words = vect.get_feature_names()
    zipped = list(zip(words, nb_clf.feature_log_prob_[class_ind]*-1))
    sorted_zip = sorted(zipped, key=lambda t: t[1], reverse=True)

    return sorted_zip

pos_all = get_salient_words(final_model['clf'], final_model['vec'], 0)

#Pickle for use in app
with open("saved-objects/irish_log_probs.p", "wb") as p:
    pickle.dump(pos_all, p)

#### Feature summary

In [48]:
# Count occurrences of each ngram combination, probabilities
df_features = pd.DataFrame(pos_all, columns=['ngram','log_prob'])
print(f"There are {df_features['ngram'].nunique()} different ngrams.")
print(f"There are {len(df_features['log_prob'].value_counts())} different probability values.")
print()

# Max and min log probabilities
print(f"Minimum feature log probability: {df_features['log_prob'].min()}")
print(f"Maximum feature log probability: {df_features['log_prob'].max()}")
print()

# View counts of log probability values
print("Occurrences of each probability value:")
print(df_features['log_prob'].value_counts())

There are 7289 different ngrams.
There are 82 different probability values.

Minimum feature log probability: -12.117126671140499
Maximum feature log probability: -4.695949142545048

Occurrences of each probability value:
-12.117127    3353
-12.117127    1247
-9.719231      784
-12.117127     273
-9.072604      237
              ... 
-5.443829        1
-7.099847        1
-4.695949        1
-5.495721        1
-5.842365        1
Name: log_prob, Length: 82, dtype: int64


In [47]:
# Save model for later
with open("saved-objects/pickled_CompNB_model.p", "wb") as p:
    pickle.dump(final_model, p)

#### (Extra Stuff) Score Testing

In [24]:
# This is the class prior log probability of an Irish name
final_model['clf'].class_log_prior_[1]

-1.4083782189053284

In [25]:
# These are the correct feature log probabilities of a feature in the Irish name class
final_model['clf'].feature_log_prob_[0]*-1

array([ -9.07260423, -12.11712667, -12.11712667, ..., -12.11712667,
       -12.11712667, -12.11712667])

In [12]:
df_features.head(10)
print('Murphy')
print(df_features[df_features['ngram']==' mur'].values)
print(df_features[df_features['ngram']=='murp'].values)
print(df_features[df_features['ngram']=='urph'].values)
print(df_features[df_features['ngram']=='rphy'].values)
print(df_features[df_features['ngram']=='phy '].values)
# Murphy prob = 0.000000000000005 (that's 14 zeros)
# 5.0e-15
# Murphy index ~6.58

# print('Bailey')
# print(df_features[df_features['ngram']==' bai'].values)
# print(df_features[df_features['ngram']=='bail'].values)
# print(df_features[df_features['ngram']=='aile'].values)
# print(df_features[df_features['ngram']=='iley'].values)
# print(df_features[df_features['ngram']=='ley '].values)
# Bailey prob = 0.000000000000000014 (that's 16 zeros)
# 1.4e-17
# Bailey 7.762
# Murphy index = 6.58/7.762 = .8477

# print('Burkson')
# print(df_features[df_features['ngram']==' bur'].values)
# print(df_features[df_features['ngram']=='burk'].values)
# print(df_features[df_features['ngram']=='urks'].values)
# print(df_features[df_features['ngram']=='rkso'].values)
# print(df_features[df_features['ngram']=='kson'].values)
# print(df_features[df_features['ngram']=='son '].values)
# Burkson 8.18
# Murphy index = 6.58/8.18 = .8044

print('Sims')
print(df_features[df_features['ngram']==' sim'].values)
print(df_features[df_features['ngram']=='sims'].values)
print(df_features[df_features['ngram']=='ims '].values)
# Sims 8.68
# Murphy index = 6.58/8.68 = .7581

# print('Ravichandran')
# print(df_features[df_features['ngram']==' rav'].values)
# print(df_features[df_features['ngram']=='ravi'].values)
# print(df_features[df_features['ngram']=='avic'].values)
# print(df_features[df_features['ngram']=='vich'].values)
# print(df_features[df_features['ngram']=='icha'].values)
# print(df_features[df_features['ngram']=='chan'].values)
# print(df_features[df_features['ngram']=='hand'].values)
# print(df_features[df_features['ngram']=='andr'].values)
# print(df_features[df_features['ngram']=='ndra'].values)
# print(df_features[df_features['ngram']=='dran'].values)
# print(df_features[df_features['ngram']=='ran '].values)
# Ravichandran 8.38
# Murphy index = 6.58/8.38 = .7852


# print('Platzfelder')
# print(df_features[df_features['ngram']==' pla'].values)
# print(df_features[df_features['ngram']=='plat'].values)
# print(df_features[df_features['ngram']=='latz'].values)
# print(df_features[df_features['ngram']=='atzf'].values)
# print(df_features[df_features['ngram']=='tzfe'].values)
# print(df_features[df_features['ngram']=='zfel'].values)
# print(df_features[df_features['ngram']=='feld'].values)
# print(df_features[df_features['ngram']=='elde'].values)
# print(df_features[df_features['ngram']=='lder'].values)
# print(df_features[df_features['ngram']=='der '].values)

# print('Timothy')
# print(df_features[df_features['ngram']==' tim'].values)
# print(df_features[df_features['ngram']=='timo'].values)
# print(df_features[df_features['ngram']=='imot'].values)
# print(df_features[df_features['ngram']=='moth'].values)
# print(df_features[df_features['ngram']=='othy'].values)
# print(df_features[df_features['ngram']=='thy '].values)
# Timothy 8.33
# Murphy index 6.58/8.33 = .7899

# Murphy index could be Avg log probability of a name divided by the avg log probability of the name Murphy


Murphy
[[' mur' -5.4565515213007565]]
[['murp' -6.515007850260742]]
[['urph' -6.515007850260742]]
[['rphy' -6.515007850260742]]
[['phy ' -6.377333758961208]]
Sims
[[' sim' -12.117126671140428]]
[['sims' -12.117126671140445]]
[['ims ' -12.117126671140445]]


In [21]:
final_model.predict_log_proba(['Knickerbocker'])

array([[-1.84612043e-03, -6.29559182e+00]])

In [19]:
final_model.predict_proba(['Sims'])

array([[9.99966688e-01, 3.33115689e-05]])